In [1]:
import numpy as np 
import pandas as pd
import re 
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r"C:\Users\S1130866\Desktop\Product_Automate\product.csv")
products = [i for i in set(df.Products.str.lower().tolist()) if not isinstance(i, (int, float)) and i != '0']

In [6]:
## Reading the main file that was created from Final_File_Import_Export_Version2

df_main = pd.read_excel(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\Compiled_File\Main.xlsx")


In [7]:
df_main.head()

,Type,Date,IMPORTER/EXPORTER NAME,Country,PRODUCT DESCRIPTION,QTY,UNIT,UNIT PRICE (USD),Total_VALUE_USD,INVOICE CURRENCY,SUPPLIER/BUYER NAME,Molecule Name,Product Name,AI
0,Import,2021-01-15,Krishi rasayan exports private limited,China,EMAMECTIN BENZOATE 95% TECHNICAL.ACTUAL USER R...,3.00,MT,168000.000000,504000.0000,USD,HUNAN GOFAR FINE CHEMICAL INDUSTR,EMAMECTIN,emamectin TECH,emamectin tech
1,Import,2021-01-28,Willowood chemicals pvt ltd,China,EMAMECTIN BENZOATE 95% TECH (CIB NO. CIR-1976/...,5.00,MT,155240.197960,776200.9898,CNY,WILLOWOOD HANGZHOU CO LIMITED,EMAMECTIN,emamectin TECH,emamectin tech
2,Import,2021-02-05,Syngenta,Austria,EVICENT 50 KG(EMAMECTIN BENZOATE 5% W/W+LUFENU...,1.80,MT,190296.298833,342533.3379,INR,SYNGENTA ASIA PACIFIC PRIVATE LIMITED,EMAMECTIN,"emamectin, lufenuron Others","emamectin, lufenuron Others"
3,Import,2021-02-19,Syngenta,Austria,EVICENT 50 KG(EMAMECTIN BENZOATE 5% W/W+LUFENU...,2.25,MT,190554.502756,428747.6312,INR,SYNGENTA ASIA PACIFIC PRIVATE LIMITED,EMAMECTIN,"emamectin, lufenuron Others","emamectin, lufenuron Others"
4,Import,2021-03-04,Crystal,China,EMAMECTIN BENZOATE TECHNICAL 95% (MIN) (CIB NO...,3.00,MT,228500.000000,685500.0000,USD,HEBEI VEYONG BIO CHEMICAL CO LIMITED,EMAMECTIN,emamectin TECH,emamectin tech


In [132]:
df_imp_jan = pd.read_excel(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\File to be worked on\January 2023\Import\38_Imp_TY2_Jan-23.xlsx")
df_imp_feb = pd.read_excel(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\File to be worked on\February 2023\Import\38_Imp_TY2_Feb-23.xlsx")
df_exp_jan = pd.read_excel(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\File to be worked on\January 2023\Export\38_Exp_TY2_Jan-23.xlsx")
df_exp_feb = pd.read_excel(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\File to be worked on\February 2023\Export\38_Exp_TY2_Feb-23.xlsx")

In [134]:
df_imp_jan.shape

(51941, 25)

In [141]:
df2 = pd.concat([df_exp_jan,df_exp_feb],axis = 0)
df3 = pd.concat([df_imp_jan,df_imp_feb],axis = 0)


In [142]:
df3.shape

(103459, 25)

 ### --------------------------------------Taking input from the user for the Export file----------------------------------------------------------------

# Taking the file path input from the user 

file_path = input('Enter the 38 Series Export file path: ')

# e.g. C:\Users\Bob\Desktop\example.txt
# or /home/Bob/Desktop/example.txt
print(file_path)

if os.path.exists(file_path):
    print('The file exists')

    with open(file_path, 'r', encoding="latin-1") as f:
        lines = f.readlines()

else:
    print('The specified file does NOT exist')
    

## Reading the file
df2 = pd.read_excel(file_path)

In [143]:
# Taking the file path input from the user 
# e.g. C:\Users\Bob\Desktop\example.txt
# or /home/Bob/Desktop/example.txt





def find_prod(x):
   
        #a = ('azoxystrobin')
        if re.search('R&D|RESEARCH|FREE SAMPLE|TESTING|SAMPLE|research|r&d|free|sample|For Lab use|lab use|LAB USE|FOR LAB USE',x) :
             return ""
        temp_products = list(set([i for i in products if i in re.split(' |\(|\)|\+|\\|/|\*|-|\s|\d',x.lower())]))
        if re.search('[2][-,\s]*[4][-\s]*[d]',x) :
             temp_products += [map_24d[re.search('[2][-,\s]*[4][-\s]*[d]',x)[0]]]
             #if re.search('AZOXY.{,50}|azoxy.{,50}',x):
              #temp_products += a
        if re.search('POTASH|potash|NPS|nps|PHOSPHATE|phosphate|SULFATE|sulfate|SULPHATE|sulphate',x):
             if len(temp_products)<1:
                    temp_products += 'fertilizer'
        if len(temp_products)>1 :
            temp_products = [i for i in temp_products if i!='pesticide']
            temp_products = [i for i in temp_products if i!='pesticides']
            temp_products = [i for i in temp_products if i!='rynaxypyr']
        return temp_products

    ##Mapping the Product Description column as string as sometimes it may not be read as a string
df2['Goods Description'] = df2['Goods Description'].map(str)
    
print("Working on Product Mapping")
temp = df2['Goods Description'].apply(lambda x: find_prod(x))
    # temp = temp.apply(lambda x:pesticide_check(x))

temp_processed = temp.apply(lambda x: ", ".join(x) if len(x)>=1 else "")
    
print("Product Mapping done , working on other conditions")

    ## Checking for the condition where UNit is GMS and quantity is one is not detected and anything with 0 quantity is also not detected
df2.loc[np.logical_and(df2['Unit'].gt('GMS'), np.less_equal(df2['Quantity'], 1)), 'Product'] = ''
df2.loc[df2["Quantity"]==0, "Product"] = ""
    
    ## Replacing the values that are returned as otherwise to the way we want it(The requirement is explained in the req doc)
temp_processed.replace('a, z, o, x, y, s, t, r, o, b, i, n','azoxystrobin',inplace = True)
temp_processed.replace('f, e, r, t, i, l, i, z, e, r','fertilizer',inplace = True)
temp_processed.replace('mancozebe','mancozeb',inplace = True)
temp_processed.replace('amoniusaltglyphosate71','glyphosate',inplace = True)
temp_processed.replace('AMONIUSALTGLYPHOSATE7171','glyphosate',inplace = True)
temp_processed.replace('rynaxypyr','chlorantraniliprole',inplace = True)
temp_processed.replace('glyphosatee','glyphosate',inplace = True)
temp_processed.replace('','Others',inplace = True)
    
df2['Product'] = temp_processed
    ## Filling up the blank values with 'Others'
df2['Product'] = df2['Product'].fillna('Others')
    
print("Product Mapping done , working on Suffix creation")
    
    #### CREATING THE SUFFIX COLUMN
    
# Creating a function for the same
def suf(x):
        temp_suffix = [] 
        if re.search('tech.{,50}|TECH.{,50}',x):
            temp_suffix = "TECH"
        elif re.search('IPA SALT|ipa salt',x):
            temp_suffix = "IPA SALT"
        elif re.search('62% IPA SALT|62% ipa salt',x):
            temp_suffix = "62% IPA SALT"
        elif re.search('41% SL|41% sl',x):
            temp_suffix = '41% SL'
        elif re.search('ALION PLUS.{50}|alion plus.{50}',x):
            temp_suffix = "ALION PLUS"
        elif re.search('95%',x):
            if len(list(df2['Product'].str.split(', '))) == 1:
                temp_suffix = "TECH"
            elif re.search('TC',x): 
                temp_suffix = "TECH"
            else :
                temp_suffix = "Others"  
        else :
            temp_suffix = "Others"
        return temp_suffix
    
    
df2['suffix'] =  df2['Goods Description'].apply(lambda x: suf(x))
print("Suffix Creation done")

Working on Product Mapping
Product Mapping done , working on other conditions
Product Mapping done , working on Suffix creation
Suffix Creation done


In [148]:
### Block for altering the entire data into the required format

temp2 = df2.drop(['Declaration No', 'HS Four Digit', 'HS Code','Destination Port','Indian Port','Address', 'City','Consinee Address','Port Code', 'IEC Phone', 'IEC Email', 'IEC Contact'],axis = 1)
temp2['Product Name'] = temp2['Product'] + " " + temp2['suffix']
temp2 = temp2.drop('suffix',axis = 1)
temp2['Type'] = 'Export'
temp2.rename(columns = {'Exporter Name':'IMPORTER/EXPORTER NAME','Country Origin':'Country','Goods Description':'PRODUCT DESCRIPTION','Quantity':'QTY','Unit':'UNIT','Unit Value Usd':'UNIT PRICE (USD)','Fob Usd':'Total_VALUE_USD','Currency':'INVOICE CURRENCY','Consinee Name':'SUPPLIER/BUYER NAME','Product':'Molecule Name'},inplace = True)


temp2 = temp2.loc[:,["Type","Date","IMPORTER/EXPORTER NAME","Country","PRODUCT DESCRIPTION","QTY","UNIT","UNIT PRICE (USD)","Total_VALUE_USD","INVOICE CURRENCY","SUPPLIER/BUYER NAME","Molecule Name","Product Name"]]
temp2['Product Name'].replace('Others Others','Others',inplace = True)

In [150]:
# Stacking the Export data into the main file

df_compiled = pd.concat([df_main,temp2],axis = 0 )

In [151]:
df_compiled.head()

,Type,Date,IMPORTER/EXPORTER NAME,Country,PRODUCT DESCRIPTION,QTY,UNIT,UNIT PRICE (USD),Total_VALUE_USD,INVOICE CURRENCY,SUPPLIER/BUYER NAME,Molecule Name,Product Name
0,Import,2021-01-15,KRISHI RASAYAN EXPORTS PRIVATE LIMITED,CHINA,EMAMECTIN BENZOATE 95% TECHNICAL.ACTUAL USER R...,3000.0,KGS,168.000000,504000.0000,USD,HUNAN GOFAR FINE CHEMICAL INDUSTR,EMAMECTIN,emamectin TECH
1,Import,2021-01-28,WILLOWOOD CHEMICALS PVT LTD,CHINA,EMAMECTIN BENZOATE 95% TECH (CIB NO. CIR-1976/...,5000.0,KGS,155.240198,776200.9898,CNY,WILLOWOOD HANGZHOU CO LIMITED,EMAMECTIN,emamectin TECH
2,Import,2021-02-05,SYNGENTA INDIA LIMITED,AUSTRIA,EVICENT 50 KG(EMAMECTIN BENZOATE 5% W/W+LUFENU...,1800.0,KGS,190.296299,342533.3379,INR,SYNGENTA ASIA PACIFIC PRIVATE LIMITED,EMAMECTIN,"emamectin, lufenuron Others"
3,Import,2021-02-19,SYNGENTA INDIA LIMITED,AUSTRIA,EVICENT 50 KG(EMAMECTIN BENZOATE 5% W/W+LUFENU...,2250.0,KGS,190.554503,428747.6312,INR,SYNGENTA ASIA PACIFIC PRIVATE LIMITED,EMAMECTIN,"emamectin, lufenuron Others"
4,Import,2021-03-04,CRYSTAL CROP PROTECTION LIMITED,CHINA,EMAMECTIN BENZOATE TECHNICAL 95% (MIN) (CIB NO...,3000.0,KGS,228.500000,685500.0000,USD,HEBEI VEYONG BIO CHEMICAL CO LIMITED,EMAMECTIN,emamectin TECH


### --------------------------------------Taking input from the user for the Import file---------------------------------------------------------

# Taking the file path input from the user 

file_path = input('Enter the 38 Series Import file path: ')

# e.g. C:\Users\Bob\Desktop\example.txt
# or /home/Bob/Desktop/example.txt
print(file_path)

if os.path.exists(file_path):
    print('The file exists')

    with open(file_path, 'r', encoding="latin-1") as f:
        lines = f.readlines()

else:
    print('The specified file does NOT exist')



## Reading the file
df3= pd.read_excel(file_path)

In [152]:



def find_prod(x):
        
        
    
        #a = ('azoxystrobin')
        if re.search('R&D|RESEARCH|FREE SAMPLE|TESTING|SAMPLE|research|r&d|free|sample',x) :
             return ""
        temp_products = list(set([i for i in products if i in re.split(' |\(|\)|\+|\\|/|\*|-|\s|\d',x.lower())]))
        if re.search('[2][-,\s]*[4][-\s]*[d]',x) :
             temp_products += [map_24d[re.search('[2][-,\s]*[4][-\s]*[d]',x)[0]]]
             #if re.search('AZOXY.{,50}|azoxy.{,50}',x):
              #temp_products += a
        if re.search('POTASH|potash|NPS|nps|PHOSPHATE|phosphate|SULFATE|sulfate|SULPHATE|sulphate',x):
             if len(temp_products)<1:
                    temp_products += 'fertilizer'
        if len(temp_products)>1 :
            temp_products = [i for i in temp_products if i!='pesticide']
            temp_products = [i for i in temp_products if i!='pesticides']
            temp_products = [i for i in temp_products if i!='rynaxypyr']
        return temp_products

    ##Mapping the Product Description column as string as sometimes it may not be read as a string
df3['Goods Description'] = df3['Goods Description'].map(str)
    
print("Working on Product Mapping")
temp = df3['Goods Description'].apply(lambda x: find_prod(x))
    # temp = temp.apply(lambda x:pesticide_check(x))

temp_processed = temp.apply(lambda x: ", ".join(x) if len(x)>=1 else "")
    
print("Product Mapping done , working on other conditions")

    ## Checking for the condition where UNit is GMS and quantity is one is not detected and anything with 0 quantity is also not detected
df3.loc[np.logical_and(df3['Unit'].gt('GMS'), np.less_equal(df3['Quantity'], 1)), 'Product'] = ''
df3.loc[df3["Quantity"]==0, "Product"] = ""
    
    ## Replacing the values that are returned as otherwise to the way we want it(The requirement is explained in the req doc)
temp_processed.replace('a, z, o, x, y, s, t, r, o, b, i, n','azoxystrobin',inplace = True)
temp_processed.replace('f, e, r, t, i, l, i, z, e, r','fertilizer',inplace = True)
temp_processed.replace('mancozebe','mancozeb',inplace = True)
temp_processed.replace('amoniusaltglyphosate71','glyphosate',inplace = True)
temp_processed.replace('AMONIUSALTGLYPHOSATE7171','glyphosate',inplace = True)
temp_processed.replace('rynaxypyr','chlorantraniliprole',inplace = True)
temp_processed.replace('glyphosatee','glyphosate',inplace = True)
temp_processed.replace('','Others',inplace = True)
    
df3['Product'] = temp_processed
    ## Filling up the blank values with 'Others'
df3['Product'] = df3['Product'].fillna('Others')
    
print("Product Mapping done , working on Suffix creation")
    
    #### CREATING THE SUFFIX COLUMN
    
    # Creating a function for the same
def suf(x):
        temp_suffix = [] 
        if re.search('tech.{,50}|TECH.{,50}',x):
            temp_suffix = "TECH"
        elif re.search('IPA SALT|ipa salt',x):
            temp_suffix = "IPA SALT"
        elif re.search('62% IPA SALT|62% ipa salt',x):
            temp_suffix = "62% IPA SALT"
        elif re.search('41% SL|41% sl',x):
            temp_suffix = '41% SL'
        elif re.search('ALION PLUS.{50}|alion plus.{50}',x):
            temp_suffix = "ALION PLUS"
        elif re.search('95%',x):
            if len(list(df3['Product'].str.split(', '))) == 1:
                temp_suffix = "TECH"
            elif re.search('TC',x): 
                temp_suffix = "TECH"
            else :
                temp_suffix = "Others" 
        else :
            temp_suffix = "Others"
        return temp_suffix
    
    
df3['suffix'] =  df3['Goods Description'].apply(lambda x: suf(x))
print("Suffix Creation done")


Working on Product Mapping
Product Mapping done , working on other conditions
Product Mapping done , working on Suffix creation
Suffix Creation done


In [153]:
### Block for altering the entire data int|o the required format


temp3 = df3.drop(['Port Code', 'Indian Port', 'Declaration No','Address', 'City', 'CHA No', 'CHA Name','HS Code','Total_Duty_Usd', 'Loading Port','Supplier Address', 'BE Type', 'IEC Phone','IEC Email', 'IEC Contact'],axis = 1)
temp3['Product Name'] = temp3['Product'] + " " + temp3['suffix']
temp3 = temp3.drop('suffix',axis = 1)
temp3['Type'] = 'Import'
temp3.rename(columns = {'Importer Name':'IMPORTER/EXPORTER NAME','Country Origin':'Country','Goods Description':'PRODUCT DESCRIPTION','Quantity':'QTY','Unit':'UNIT','Unit Price Usd':'UNIT PRICE (USD)','Total Value Usd':'Total_VALUE_USD','Inv Currency':'INVOICE CURRENCY','Supplier Name':'SUPPLIER/BUYER NAME','Product':'Molecule Name'},inplace = True)


temp3 = temp3.loc[:,["Type","Date","IMPORTER/EXPORTER NAME","Country","PRODUCT DESCRIPTION","QTY","UNIT","UNIT PRICE (USD)","Total_VALUE_USD","INVOICE CURRENCY","SUPPLIER/BUYER NAME","Molecule Name","Product Name"]]
temp3['Product Name'].replace('Others Others','Others',inplace = True)

In [190]:
# Stacking the Export data into the main file

df_final_compiled = pd.concat([df_compiled ,temp3],axis = 0 )

In [191]:
df_final_compiled.head()

,Type,Date,IMPORTER/EXPORTER NAME,Country,PRODUCT DESCRIPTION,QTY,UNIT,UNIT PRICE (USD),Total_VALUE_USD,INVOICE CURRENCY,SUPPLIER/BUYER NAME,Molecule Name,Product Name
0,Import,2021-01-15,KRISHI RASAYAN EXPORTS PRIVATE LIMITED,CHINA,EMAMECTIN BENZOATE 95% TECHNICAL.ACTUAL USER R...,3000.0,KGS,168.000000,504000.0000,USD,HUNAN GOFAR FINE CHEMICAL INDUSTR,EMAMECTIN,emamectin TECH
1,Import,2021-01-28,WILLOWOOD CHEMICALS PVT LTD,CHINA,EMAMECTIN BENZOATE 95% TECH (CIB NO. CIR-1976/...,5000.0,KGS,155.240198,776200.9898,CNY,WILLOWOOD HANGZHOU CO LIMITED,EMAMECTIN,emamectin TECH
2,Import,2021-02-05,SYNGENTA INDIA LIMITED,AUSTRIA,EVICENT 50 KG(EMAMECTIN BENZOATE 5% W/W+LUFENU...,1800.0,KGS,190.296299,342533.3379,INR,SYNGENTA ASIA PACIFIC PRIVATE LIMITED,EMAMECTIN,"emamectin, lufenuron Others"
3,Import,2021-02-19,SYNGENTA INDIA LIMITED,AUSTRIA,EVICENT 50 KG(EMAMECTIN BENZOATE 5% W/W+LUFENU...,2250.0,KGS,190.554503,428747.6312,INR,SYNGENTA ASIA PACIFIC PRIVATE LIMITED,EMAMECTIN,"emamectin, lufenuron Others"
4,Import,2021-03-04,CRYSTAL CROP PROTECTION LIMITED,CHINA,EMAMECTIN BENZOATE TECHNICAL 95% (MIN) (CIB NO...,3000.0,KGS,228.500000,685500.0000,USD,HEBEI VEYONG BIO CHEMICAL CO LIMITED,EMAMECTIN,emamectin TECH


In [192]:
df_final_compiled = df_final_compiled[df_final_compiled['Molecule Name'] != 'Others']

df_final_compiled2 = df_final_compiled

In [193]:
df_final_compiled2.shape

(175107, 13)

In [194]:
#### -------------------------------  WORKING ON THE COUNTRY, COMPANY , QTY and Molecule Name Columns  -------------------------



#### Working on the compnay column i,e IMPORTER/EXPORTER NAME

company = pd.read_excel(r'C:\Users\S1130866\Desktop\Product_Automate\Country and company.xlsx')
country = pd.read_excel(r'C:\Users\S1130866\Desktop\Product_Automate\Country and company.xlsx',sheet_name = "Country") 

unique = company.drop_duplicates()
unique_mismatches = unique[unique['Company'].str.lower() != unique['New'].str.lower()]
company_key_value = dict(zip(unique_mismatches['Company'].str.lower(),unique_mismatches['New'].str.lower()))
df_final_compiled2['IMPORTER/EXPORTER NAME'] = df_final_compiled2['IMPORTER/EXPORTER NAME'].str.lower().replace(company_key_value)



### Working on the country column



unique2 = country.drop_duplicates()
unique_mismatches2 = unique2[unique2['Country'].str.lower() != unique2['New'].str.lower()]
country_key_value = dict(zip(unique_mismatches2['Country'].str.lower(),unique_mismatches2['New'].str.lower()))
df_final_compiled2['Country'] = df_final_compiled2['Country'].str.lower().replace(country_key_value)







In [195]:
### -------------------------------  UNIT COLUMN -------------------------------------------

df_final_compiled3 = df_final_compiled2

# df_final_compiled3['UNIT'] =df_final_compiled3['UNIT'].str.rstrip().replace(unit_dict)



df_final_compiled3['UNIT'] = df_final_compiled3['UNIT'].apply(lambda x: x.strip())




#1l = 1kg

In [181]:
df_final_compiled3.head()

,Type,Date,IMPORTER/EXPORTER NAME,Country,PRODUCT DESCRIPTION,QTY,UNIT,UNIT PRICE (USD),Total_VALUE_USD,INVOICE CURRENCY,SUPPLIER/BUYER NAME,Molecule Name,Product Name
0,Import,2021-01-15,krishi rasayan exports private limited,china,EMAMECTIN BENZOATE 95% TECHNICAL.ACTUAL USER R...,3000.0,KGS,168.000000,504000.0000,USD,HUNAN GOFAR FINE CHEMICAL INDUSTR,EMAMECTIN,emamectin TECH
1,Import,2021-01-28,willowood chemicals pvt ltd,china,EMAMECTIN BENZOATE 95% TECH (CIB NO. CIR-1976/...,5000.0,KGS,155.240198,776200.9898,CNY,WILLOWOOD HANGZHOU CO LIMITED,EMAMECTIN,emamectin TECH
2,Import,2021-02-05,syngenta,austria,EVICENT 50 KG(EMAMECTIN BENZOATE 5% W/W+LUFENU...,1800.0,KGS,190.296299,342533.3379,INR,SYNGENTA ASIA PACIFIC PRIVATE LIMITED,EMAMECTIN,"emamectin, lufenuron Others"
3,Import,2021-02-19,syngenta,austria,EVICENT 50 KG(EMAMECTIN BENZOATE 5% W/W+LUFENU...,2250.0,KGS,190.554503,428747.6312,INR,SYNGENTA ASIA PACIFIC PRIVATE LIMITED,EMAMECTIN,"emamectin, lufenuron Others"
4,Import,2021-03-04,crystal,china,EMAMECTIN BENZOATE TECHNICAL 95% (MIN) (CIB NO...,3000.0,KGS,228.500000,685500.0000,USD,HEBEI VEYONG BIO CHEMICAL CO LIMITED,EMAMECTIN,emamectin TECH


In [196]:

df_final_compiled3['UNIT PRICE (USD)'] = df_final_compiled3.apply(lambda x: x['UNIT PRICE (USD)'] * 1000 if x['UNIT']== 'KGS' else x['UNIT PRICE (USD)'], axis=1)
df_final_compiled3['UNIT PRICE (USD)'] = df_final_compiled3.apply(lambda x: x['UNIT PRICE (USD)'] * 1000 if x['UNIT']== 'LTR' else x['UNIT PRICE (USD)'], axis=1)
df_final_compiled3['UNIT PRICE (USD)'] = df_final_compiled3.apply(lambda x: x['UNIT PRICE (USD)'] * 1000 if x['UNIT']== 'LT/KG' else x['UNIT PRICE (USD)'], axis=1)
df_final_compiled3['UNIT PRICE (USD)'] = df_final_compiled3.apply(lambda x: x['UNIT PRICE (USD)'] * 1 if x['UNIT']== 'MTS' else x['UNIT PRICE (USD)'], axis=1)
df_final_compiled3['UNIT PRICE (USD)'] = df_final_compiled3.apply(lambda x: x['UNIT PRICE (USD)'] * 1 if x['UNIT']== 'MLT' else x['UNIT PRICE (USD)'], axis=1)
df_final_compiled3['UNIT PRICE (USD)'] = df_final_compiled3.apply(lambda x: x['UNIT PRICE (USD)'] * 1000000 if x['UNIT']== 'MGS' else x['UNIT PRICE (USD)'], axis=1)
df_final_compiled3['UNIT PRICE (USD)'] = df_final_compiled3.apply(lambda x: x['UNIT PRICE (USD)'] * 1000000 if x['UNIT']== 'GMS' else x['UNIT PRICE (USD)'], axis=1)
df_final_compiled3['UNIT PRICE (USD)'] = df_final_compiled3.apply(lambda x: x['UNIT PRICE (USD)'] * 2204.62442018 if x['UNIT']== 'LBS' else x['UNIT PRICE (USD)'], axis=1)




In [197]:
df_final_compiled3.head()

,Type,Date,IMPORTER/EXPORTER NAME,Country,PRODUCT DESCRIPTION,QTY,UNIT,UNIT PRICE (USD),Total_VALUE_USD,INVOICE CURRENCY,SUPPLIER/BUYER NAME,Molecule Name,Product Name
0,Import,2021-01-15,krishi rasayan exports private limited,china,EMAMECTIN BENZOATE 95% TECHNICAL.ACTUAL USER R...,3000.0,KGS,168000.000000,504000.0000,USD,HUNAN GOFAR FINE CHEMICAL INDUSTR,EMAMECTIN,emamectin TECH
1,Import,2021-01-28,willowood chemicals pvt ltd,china,EMAMECTIN BENZOATE 95% TECH (CIB NO. CIR-1976/...,5000.0,KGS,155240.197960,776200.9898,CNY,WILLOWOOD HANGZHOU CO LIMITED,EMAMECTIN,emamectin TECH
2,Import,2021-02-05,syngenta,austria,EVICENT 50 KG(EMAMECTIN BENZOATE 5% W/W+LUFENU...,1800.0,KGS,190296.298833,342533.3379,INR,SYNGENTA ASIA PACIFIC PRIVATE LIMITED,EMAMECTIN,"emamectin, lufenuron Others"
3,Import,2021-02-19,syngenta,austria,EVICENT 50 KG(EMAMECTIN BENZOATE 5% W/W+LUFENU...,2250.0,KGS,190554.502756,428747.6312,INR,SYNGENTA ASIA PACIFIC PRIVATE LIMITED,EMAMECTIN,"emamectin, lufenuron Others"
4,Import,2021-03-04,crystal,china,EMAMECTIN BENZOATE TECHNICAL 95% (MIN) (CIB NO...,3000.0,KGS,228500.000000,685500.0000,USD,HEBEI VEYONG BIO CHEMICAL CO LIMITED,EMAMECTIN,emamectin TECH


In [198]:



df_final_compiled3['QTY'] = df_final_compiled3.apply(lambda x: x['QTY'] * 0.001 if x['UNIT']== 'KGS' else x['QTY'], axis=1)
df_final_compiled3['QTY'] = df_final_compiled3.apply(lambda x: x['QTY'] * 0.001 if x['UNIT']== 'LTR' else x['QTY'], axis=1)
df_final_compiled3['QTY'] = df_final_compiled3.apply(lambda x: x['QTY'] * 0.001 if x['UNIT']== 'LT/KG' else x['QTY'], axis=1)
df_final_compiled3['QTY'] = df_final_compiled3.apply(lambda x: x['QTY'] * 1 if x['UNIT']== 'MTS' else x['QTY'], axis=1)
df_final_compiled3['QTY'] = df_final_compiled3.apply(lambda x: x['QTY'] * 1 if x['UNIT']== 'MLT' else x['QTY'], axis=1)
df_final_compiled3['QTY'] = df_final_compiled3.apply(lambda x: x['QTY'] * 0.000001 if x['UNIT']== 'MGS' else x['QTY'], axis=1)
df_final_compiled3['QTY'] = df_final_compiled3.apply(lambda x: x['QTY'] * 0.000001 if x['UNIT']== 'GMS' else x['QTY'], axis=1)
df_final_compiled3['QTY'] = df_final_compiled3.apply(lambda x: x['QTY'] * 0.000453592 if x['UNIT']== 'LBS' else x['QTY'], axis=1)














## Values to be converted into the required format 

"""

'KGS':'KG/LT',
'KGS':'KG/LT ,
'GMS':'GMs/ML' ,
'GMS':'GMs/ML' ,
'LT/KG':'KG/LT'
'LTR':'KG/LT',
'LTR':'KG/LT',
'MTS':'Metric Tons' ,
'MTS':'Metric Tons' ,
'MLT':'Metric LT '
'MGS':'GMs/ML' ,
'MGS':'GMs/ML'


"""



unit_dict_Other = {'BAG':'Other',
             'BTL':'Other', 
             'CAN':'Other',
             'CBM':'Other',
             'CTM':'Other',
             'DOZ':'Other',
             'DRM':'Other',
'GRS':'Other',
'KGA':'Other',
'KIT':'Other',
'KLR':'Other',
'MTR':'Other',
'MTR':'Other',
'NOS':'Other',
'NOS':'Other',
'PAC':'Other',
'PCS':'Other',
'PCS':'Other',
'PRS':'Other',
'RLS':'Other',
'SET':'Other',
'SET':'Other',
'SQM':'Other',
'THD':'Other',
'TUB':'Other',
'UNT':'Other',
'UNT':'Other',
'VLS':'Other',
'VLS':'Other',
'TBS':'Other',
'BRL':'Other',
'KGS':'MT',
'GMS':'MT',
'LT/KG':'MT',
'LTR':'MT',
'MTS':'MT',
'MLT':'MT',
'MGS':'MT',
'LBS':'MT',
'MT':'MT',
'Other':'Other'
}



df_final_compiled3['UNIT'] = df_final_compiled3['UNIT'].map(unit_dict_Other)



In [199]:
df_final_compiled3['UNIT'].value_counts()

MT       126046
Other     49061
Name: UNIT, dtype: int64

In [185]:
###  ----------- ORDER FOR AI's  --------------------

AI_Order = pd.read_excel(r"C:\Users\S1130866\Desktop\Product_Automate\Tech vs Other.xlsx")


AI_Mapping = dict(zip(AI_Order['Product Name 1'].str.lower(),AI_Order['Product Name 2'].str.lower()))

AI_Mapping

def AI_MAP(x) :
    a = 'Others'
    if x in AI_Mapping.keys():
        x.map(AI_Mapping)
    else  : 
        elements =  [i for i in x.split(', ')]
        if len(elements)>1:
            return str(a)
        else : 
            return x

df_final_compiled3['AI'] = df_final_compiled3['Product Name'].apply(lambda x : x.lower())

df_final_compiled3['AI'] = df_final_compiled3['AI'].map(AI_Mapping).fillna(df_final_compiled3['Product Name'])



df_final_compiled3['Country'] = df_final_compiled3['Country'].str.capitalize()
df_final_compiled3['IMPORTER/EXPORTER NAME'] = df_final_compiled3['IMPORTER/EXPORTER NAME'].str.capitalize()


    


df_final_compiled4 = df_final_compiled3.loc[df_final_compiled3['UNIT'] == 'MT']





In [186]:
df_final_compiled3.to_excel(r"C:\Users\S1130866\OneDrive - Syngenta\Documents\Shared\Product_Automate\Product_Automate\Import_Export\Compiled_File\Main.xlsx",sheet_name = 'Data',index = False)

In [187]:
df_final_compiled3.head()

,Type,Date,IMPORTER/EXPORTER NAME,Country,PRODUCT DESCRIPTION,QTY,UNIT,UNIT PRICE (USD),Total_VALUE_USD,INVOICE CURRENCY,SUPPLIER/BUYER NAME,Molecule Name,Product Name,AI
0,Import,2021-01-15,Krishi rasayan exports private limited,China,EMAMECTIN BENZOATE 95% TECHNICAL.ACTUAL USER R...,3.00,MT,168000.000000,504000.0000,USD,HUNAN GOFAR FINE CHEMICAL INDUSTR,EMAMECTIN,emamectin TECH,emamectin tech
1,Import,2021-01-28,Willowood chemicals pvt ltd,China,EMAMECTIN BENZOATE 95% TECH (CIB NO. CIR-1976/...,5.00,MT,155240.197960,776200.9898,CNY,WILLOWOOD HANGZHOU CO LIMITED,EMAMECTIN,emamectin TECH,emamectin tech
2,Import,2021-02-05,Syngenta,Austria,EVICENT 50 KG(EMAMECTIN BENZOATE 5% W/W+LUFENU...,1.80,MT,190296.298833,342533.3379,INR,SYNGENTA ASIA PACIFIC PRIVATE LIMITED,EMAMECTIN,"emamectin, lufenuron Others","emamectin, lufenuron Others"
3,Import,2021-02-19,Syngenta,Austria,EVICENT 50 KG(EMAMECTIN BENZOATE 5% W/W+LUFENU...,2.25,MT,190554.502756,428747.6312,INR,SYNGENTA ASIA PACIFIC PRIVATE LIMITED,EMAMECTIN,"emamectin, lufenuron Others","emamectin, lufenuron Others"
4,Import,2021-03-04,Crystal,China,EMAMECTIN BENZOATE TECHNICAL 95% (MIN) (CIB NO...,3.00,MT,228500.000000,685500.0000,USD,HEBEI VEYONG BIO CHEMICAL CO LIMITED,EMAMECTIN,emamectin TECH,emamectin tech


In [189]:
df_final_compiled3.shape

(175107, 14)

In [130]:
df_main.drop('Product',axis = 1 , inplace = True)

In [15]:
df_main.columns

Index(['Type', 'Date', 'IMPORTER/EXPORTER NAME', 'Country',
       'PRODUCT DESCRIPTION', 'QTY', 'UNIT', 'UNIT PRICE (USD)',
       'Total_VALUE_USD', 'INVOICE CURRENCY', 'SUPPLIER/BUYER NAME',
       'Molecule Name', 'Product Name', 'AI'],
      dtype='object')